# Artificial Neural Networks

This week, we will learn to implement artificial neural networks, another class of models that can be used in molecular property prediction. 

![](mldd_diagram_lab3.png)

# Molecular graphs

**Recap:** In mathematics, a graph is an object that consists of a set of vertices (nodes) connected with edges, i.e. $\mathcal{G} = (V, E)$, where $V = \{ v_i: i \in \{1, 2, \dots, N \} \}$ and $E \subseteq \{ (v_i, v_j):\, v_i,v_j \in V \}$.

Molecular graphs are a special class of graphs, where besides nodes (denoting atoms) and edges (denoting chemical bonds), we have an additional information about atom types and sometimes also bond types. We can assume that we have an additional set of node/atom features encoded as a matrix $X$, where $X_{ij}$ is the $j$-th feature of the $i$-th atom. As atomic features, we can have one-hot encoded atom symbols (a vector containing zeros on all positions besides the position that corresponds to the atom symbol), the number of implicit hydrogens bonded with this atom, or the number of heavy neighbors (atoms other than hydrogens bonded to the given atom).

Egdes/bonds can be encoded in two different ways. One method is to use an adjacency matrix $A$, where $A_{ij}=1$ if nodes/atoms $v_i$ nad $v_j$ are connected ($A_{ij}=0$ otherwise). In the case of sparse matrices, a more useful encoding is a list of pairs of connected atoms (a list of index pairs). This latter enocding is used by the PyTorch-Geometric library.

In practice, a molecular graph can be described by two matrices: $X \in \mathbb{R}^{N \times F}$ and $E \in \{0, 1,\dots,N-1\}^{2 \times N}$, where $N$ is the number of atoms, and $F$ is the number of atomic features.

![molecular graph](../../lectures/assets/mol_graph.png)

**Exercise 2:** Create a molecular graph dataset using PyTorch-Geometric and the same data as in Exercise 1.

In [1]:
import pandas as pd
import numpy as np
import torch
from tdc.single_pred.adme import ADME
from tdc import Evaluator
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem
from matplotlib import pyplot as plt
from IPython import display

from typing import List, Tuple


class Featurizer:
    def __init__(self, y_column, smiles_col='Drug', **kwargs):
        self.y_column = y_column
        self.smiles_col = smiles_col
        self.__dict__.update(kwargs)
    
    def __call__(self, df):
        raise NotImplementedError()


class ECFPFeaturizer(Featurizer):
    def __init__(self, y_column, radius=2, length=1024, **kwargs):
        self.radius = radius
        self.length = length
        super().__init__(y_column, **kwargs)
    
    def __call__(self, df):
        fingerprints = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row[self.smiles_col]
            mol = Chem.MolFromSmiles(smiles)
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, nBits=self.length)
            fingerprints.append(fp)
            labels.append(y)
        fingerprints = np.array(fingerprints)
        labels = np.array(labels)
        return fingerprints, labels

data = ADME('Solubility_AqSolDB')
split = data.get_split()
rmse = Evaluator(name = 'RMSE')

featurizer = ECFPFeaturizer(y_column='Y')
X_train, y_train = featurizer(split['train'])
X_valid, y_valid = featurizer(split['valid'])
X_test, y_test = featurizer(split['test'])

Found local copy...
Loading...
Done!
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom without neighbors
[16:23:25] WARNING: not removing hydrogen atom withou

In [2]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise ValueError("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


class GraphFeaturizer(Featurizer):
    def __call__(self, df):
        graphs = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row[self.smiles_col]
            mol = Chem.MolFromSmiles(smiles)
            
            edges = []
            for bond in mol.GetBonds():
                edges.append((bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()))
                edges.append((bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()))  # TODO: Add edges in both directions
            edges = np.array(edges)
            
            nodes = []
            for atom in mol.GetAtoms():
                results = one_of_k_encoding_unk(atom.GetSymbol(), ['C', 'O', 'N', 'S', 'Cl', 'F', 'I', 'Br'])  # TODO: Add atom features as a list, you can use one_of_k_encodings defined above
                nodes.append(results)
            nodes = np.array(nodes)
            
            graphs.append((nodes, edges.T))
            labels.append(y)
        labels = np.array(labels)
        return [Data(
            x=torch.FloatTensor(x), 
            edge_index=torch.LongTensor(edge_index), 
            y=torch.FloatTensor([y])
        ) for ((x, edge_index), y) in zip(graphs, labels)]

In [3]:
featurizer = GraphFeaturizer('Y')
graph = featurizer(split['test'].iloc[:1])[0]

assert graph.x.ndim == 2, 'The atom features should be a matrix with dimensions (number of atoms) x (number of features)'
assert graph.edge_index.ndim == 2, 'The edges should be represented as a matrix of atom pairs'
assert graph.edge_index.shape[0] == 2, 'The first dimension should be 2 (we need atom pairs)'
assert isinstance(graph.y, torch.FloatTensor) and graph.y.shape == (1,), 'The graph label should be assigned to the variable `y`'
print('Looks OK!')

Looks OK!


---

<div style="display: flex; justify-content: space-between">
    <div style="width: 20%; display: inline-block; margin: 20px">
        <img src="../../assets/profile1.png" width="100%">
    </div>
    <div style="width: 60%; display: inline-block; margin: 20px">
        <p><strong>Nitro:</strong> Atoms possess several important properties that define their behavior in chemical systems. The <strong>atomic number</strong> of an atom signifies the number of protons in its nucleus, which determines its elemental identity and placement on the periodic table. <strong>Formal charge</strong> represents the charge assigned to an atom in a molecule or ion, calculated by considering the number of valence electrons and its bonding situation. <strong>Partial charge</strong> refers to the uneven distribution of electron density within a molecule, resulting in regions of slight positive or negative charge. <strong>Hybridization</strong> is a concept that describes the mixing of atomic orbitals to form hybrid orbitals, crucial for understanding molecular geometry and bonding patterns. These properties collectively influence an atom's reactivity, bonding behavior, and its role within chemical compounds, providing insight into the complex nature of molecular interactions.</p>
    </div>
</div>
<div style="width: 80%; display: block; margin: 0 20px 0">
    <p>In typical druglike compounds, a diverse array of atom types can be found, each playing a crucial role in the compound's pharmacological activity. <strong>Carbon</strong> atoms are ubiquitous, forming the backbone of organic molecules and providing structural stability. <strong>Hydrogen</strong> atoms frequently accompany carbon, contributing to molecular stability and participating in various bonding interactions. <strong>Oxygen</strong> atoms are common, often found in functional groups such as hydroxyl (-OH) and carbonyl (C=O), influencing the compound's polarity and reactivity. <strong>Nitrogen</strong> atoms, present in amines and amides, contribute to the compound's basicity and ability to form hydrogen bonds. Additionally, <strong>sulfur</strong> atoms may appear in thiol (-SH) or sulfide (-S-) groups, introducing potential sites for redox reactions or metal binding. Halogens, such as <strong>fluorine</strong>, <strong>chlorine</strong>, <strong>bromine</strong>, and <strong>iodine</strong>, are also commonly encountered in druglike compounds, where they can serve as substituents or functional groups, imparting specific chemical properties and influencing the compound's biological activity and metabolic stability. These diverse atom types collectively contribute to the complex pharmacological properties of druglike compounds, influencing their efficacy, safety, and pharmacokinetic profiles.</p>
    </div>

---

# Graph neural networks

Graph neural networks (GNNs) can process graph representation given at the input to the model. A graph layer uses the input atom features and graph topology to calculate new atom representations, but the graph structure is not changed (the edges stay the same). We can use the calculated atom representations for **node classification**, or we can add a graph readout operation at the end to aggregate information from all nodes into one vector that describes the whole graph (e.g. we can use the average atom representation). The graph representation can be then processed by fully-connected layers for the **graph classification** task.


## Message Passing Neural Networks (MPNN)

MPNN is a general description of a graph neural network, and many graph neural network architectures can be matched with this description (including the ones listed below). In MPNN, **messages** $M$ from the neighboring atoms are retrieved to calculate a new atom representation via the **update** $U$ operation. This procedure is repeated a few times before we use **readout** $R$ to compute the graph representation.

$$
\mathbf{m}_i^{t+1} = \sum_{j\in\mathcal{N}(i)} M_t(\mathbf{h}_i^t, \mathbf{h}_j^t, \mathbf{e}_{ij})\\
\mathbf{h}_i^{t+1} = U_t(\mathbf{h}_i^t, \mathbf{m}_i^{t+1}) \\
\hat{y} = R(\{\mathbf{h}_i^T \, | \, i \in \mathcal{G} \})
$$

- **Graph Convolutional Networks (GCN)**

$$
\mathbf{h}_i^{t+1} = W^T \sum_{j\in\mathcal{N}(i)\cup \{i\}} \frac{e_{ij}}{\sqrt{\hat{d}_i \hat{d}_j}} \mathbf{h}_j^t
$$

- **Graph Isomorphism Networks (GIN)**

$$
\mathbf{h}_i^{t+1} = W^T \left( (1+\varepsilon)\mathbf{h}_i^t + \sum_{j\in\mathcal{N}(i)} \mathbf{h}_j^t \right)
$$

- **GraphSAGE**

$$
\mathbf{h}_i^{t+1} = W_1 \mathbf{h}_i^t + W_2 \frac{1}{|\mathcal{N}(i)|} \sum_{j\in\mathcal{N}(i)} \mathbf{h}_j^t
$$

- **Graph Attention Networks (GAT)**

$$
\mathbf{h}_i^{t+1} = \sum_{j\in\mathcal{N}(i)\cup \{i\}} \alpha_{ij} W \mathbf{h}_j^t,\\
\alpha_{ij} = \frac{\exp\left( LeakyReLU(\mathbf{a}[W\mathbf{h}_i^t \| W\mathbf{h}_j^t])\right)}{\sum_{k\in\mathcal{N}(i) \cup \{i\}}\exp\left( LeakyReLU(\mathbf{a}[W\mathbf{h}_i^t \| W\mathbf{h}_k^t])\right)}
$$

**Exercise 3:** Use the molecular graphs prepared in the previous exercise to predict compound solubility.

In [4]:
import pandas as pd
import numpy as np
import torch
from tdc.single_pred.adme import ADME
from tdc import Evaluator
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem
from matplotlib import pyplot as plt
from IPython import display

from typing import List, Tuple


class Featurizer:
    def __init__(self, y_column, smiles_col='Drug', **kwargs):
        self.y_column = y_column
        self.smiles_col = smiles_col
        self.__dict__.update(kwargs)
    
    def __call__(self, df):
        raise NotImplementedError()


class ECFPFeaturizer(Featurizer):
    def __init__(self, y_column, radius=2, length=1024, **kwargs):
        self.radius = radius
        self.length = length
        super().__init__(y_column, **kwargs)
    
    def __call__(self, df):
        fingerprints = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row[self.smiles_col]
            mol = Chem.MolFromSmiles(smiles)
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, nBits=self.length)
            fingerprints.append(fp)
            labels.append(y)
        fingerprints = np.array(fingerprints)
        labels = np.array(labels)
        return fingerprints, labels

data = ADME('Solubility_AqSolDB')
split = data.get_split()
rmse = Evaluator(name = 'RMSE')

featurizer = ECFPFeaturizer(y_column='Y')
X_train, y_train = featurizer(split['train'])
X_valid, y_valid = featurizer(split['valid'])
X_test, y_test = featurizer(split['test'])

Found local copy...
Loading...
Done!
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom without neighbors
[16:23:30] WARNING: not removing hydrogen atom withou

In [5]:
from torch_geometric.loader import DataLoader as GraphDataLoader


# prepare data loaders
batch_size = 64
train_loader = GraphDataLoader(featurizer(split['train']), batch_size=batch_size, shuffle=True)
valid_loader = GraphDataLoader(featurizer(split['valid']), batch_size=batch_size)
test_loader = GraphDataLoader(featurizer(split['test']), batch_size=batch_size)

[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not removing hydrogen atom without neighbors
[16:23:34] WARNING: not r

In [6]:
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F


class GraphNeuralNetwork(torch.nn.Module):  # TODO: assign hyperparameters to attributes and define the forward pass
    def __init__(self, hidden_size):
        super().__init__()
        self.gcn1 = GCNConv(in_channels=8, out_channels=hidden_size)
        self.gcn2 = GCNConv(in_channels=hidden_size, out_channels=hidden_size)
        self.gcn3 = GCNConv(in_channels=hidden_size, out_channels=hidden_size)
        self.lin1 = torch.nn.Linear(in_features=hidden_size, out_features=1)
    
    def forward(self, x, edge_index, batch):
        y = self.gcn1(x, edge_index)
        y = F.relu(y)
        y = self.gcn2(y, edge_index)
        y = F.relu(y)
        y = self.gcn3(y, edge_index)
        y = global_mean_pool(y, batch)
        y = self.lin1(y)
        return y

In [7]:
df_logs = pd.DataFrame()

def train(train_loader, valid_loader, df_logs):
    # hyperparameters definition
    hidden_size = 512
    epochs = 10
    learning_rate = 0.0001
    
    # model preparation
    model = GraphNeuralNetwork(hidden_size)  # TODO: you can add more hyperparameters if needed
    model.train()
    
    # training loop
    optimizer = torch.optim.Adam(model.parameters()) # TODO: define an optimizer
    loss_fn = torch.nn.MSELoss()  # TODO: define a loss function

    for epoch in trange(1, epochs + 1, leave=False):
        model.train()
        for data in tqdm(train_loader, leave=False):
            x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y
            model.zero_grad()
            preds = model(x, edge_index, batch)
            loss = loss_fn(preds, y.reshape(-1, 1))
            loss.backward()
            optimizer.step()

        # validation loop
        preds = predict(model, valid_loader)
        df_logs = pd.concat([df_logs, pd.DataFrame(
            {'epoch': epoch,
             'preds': preds.flatten(),
             'mode': 'valid'
        }, index=range(len(preds.flatten())))], ignore_index=True)
    
    return model, df_logs


def predict(model, test_loader):
    # evaluation loop
    model.eval()
    preds_batches = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            
            preds = model(x, edge_index, batch)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds


# training
model, df_logs = train(train_loader, valid_loader, df_logs)

# evaluation
predictions = predict(model, test_loader)
df_logs = pd.concat([df_logs, pd.DataFrame({
    'epoch': -1,
    'preds': predictions.flatten(),
    'mode': 'test'
}, index=range(len(predictions.flatten())))], ignore_index=True)

rmse_score = rmse(y_test, predictions.flatten())

print(f'RMSE = {rmse_score:.2f}')
assert rmse_score < 1.4, "It should be possible to obtain RMSE lower than 1.4"
print('Looks OK!')
df_logs.to_csv('training_logs.csv')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: DataLoader found invalid type: '<class 'numpy.ndarray'>'

In [8]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

def rmse_np(y_true, y_pred):
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mae_np(y_true, y_pred):
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    return np.mean(np.abs(y_true - y_pred))

def predict_with_targets(model, loader):
    model.eval()
    preds_batches = []
    y_batches = []
    with torch.no_grad():
        for data in tqdm(loader, leave=False):
            x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y
            preds = model(x, edge_index, batch)
            preds_batches.append(preds.cpu().numpy())
            y_batches.append(y.cpu().numpy())
    preds = np.concatenate(preds_batches).flatten()
    y_true = np.concatenate(y_batches).flatten()
    return y_true, preds


In [9]:
def train_one_run(train_loader, valid_loader, hidden_size=256, lr=1e-3, epochs=30, weight_decay=0.0, dropout=0.0, seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = GraphNeuralNetwork(hidden_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = torch.nn.MSELoss()

    history = []

    for epoch in trange(1, epochs + 1, leave=False):
        model.train()
        losses = []

        for data in tqdm(train_loader, leave=False):
            x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y

            optimizer.zero_grad()
            preds = model(x, edge_index, batch)
            loss = loss_fn(preds, y.reshape(-1, 1))
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        train_loss = float(np.mean(losses))

        # walidacja: RMSE + MAE
        y_valid, preds_valid = predict_with_targets(model, valid_loader)
        valid_rmse = rmse_np(y_valid, preds_valid)
        valid_mae = mae_np(y_valid, preds_valid)

        history.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "valid_rmse": valid_rmse,
            "valid_mae": valid_mae
        })

    return model, pd.DataFrame(history)


In [10]:
param_grid = {
    "hidden_size": [64, 128, 256, 512],
    "lr": [1e-3, 5e-4, 1e-4],
    "epochs": [15],  # możesz dać 30
    "weight_decay": [0.0, 1e-4, 1e-3],
    "seed": [42]
}

results = []
all_histories = []

run_id = 0

for hidden_size in param_grid["hidden_size"]:
    for lr in param_grid["lr"]:
        for epochs in param_grid["epochs"]:
            for wd in param_grid["weight_decay"]:
                for seed in param_grid["seed"]:
                    run_id += 1
                    print(f"\n=== RUN {run_id}: hidden={hidden_size}, lr={lr}, epochs={epochs}, wd={wd}, seed={seed} ===")

                    model, history_df = train_one_run(
                        train_loader=train_loader,
                        valid_loader=valid_loader,
                        hidden_size=hidden_size,
                        lr=lr,
                        epochs=epochs,
                        weight_decay=wd,
                        seed=seed
                    )

                    best_row = history_df.loc[history_df["valid_rmse"].idxmin()]
                    best_valid_rmse = best_row["valid_rmse"]
                    best_epoch = int(best_row["epoch"])

                    results.append({
                        "run_id": run_id,
                        "hidden_size": hidden_size,
                        "lr": lr,
                        "epochs": epochs,
                        "weight_decay": wd,
                        "seed": seed,
                        "best_valid_rmse": best_valid_rmse,
                        "best_epoch": best_epoch
                    })

                    history_df["run_id"] = run_id
                    history_df["hidden_size"] = hidden_size
                    history_df["lr"] = lr
                    history_df["weight_decay"] = wd
                    all_histories.append(history_df)

results_df = pd.DataFrame(results).sort_values("best_valid_rmse")
histories_df = pd.concat(all_histories, ignore_index=True)

display(results_df.head(10))



=== RUN 1: hidden=64, lr=0.001, epochs=15, wd=0.0, seed=42 ===


TypeError: DataLoader found invalid type: '<class 'numpy.ndarray'>'

In [11]:
best_cfg = results_df.iloc[0].to_dict()
best_cfg


NameError: name 'results_df' is not defined

In [ ]:
best_hidden = int(best_cfg["hidden_size"])
best_lr = float(best_cfg["lr"])
best_wd = float(best_cfg["weight_decay"])

final_model, final_history = train_one_run(
    train_loader=train_loader,
    valid_loader=valid_loader,
    hidden_size=best_hidden,
    lr=best_lr,
    epochs=30,           # docelowo więcej
    weight_decay=best_wd,
    seed=42
)

display(final_history.tail())


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(final_history["epoch"], final_history["train_loss"], label="Train MSE")
plt.plot(final_history["epoch"], final_history["valid_rmse"], label="Valid RMSE")
plt.xlabel("Epoch")
plt.ylabel("Metric")
plt.title("Training Curve (MSE vs RMSE)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
y_test_true, y_test_pred = predict_with_targets(final_model, test_loader)

test_rmse = rmse_np(y_test_true, y_test_pred)
test_mae = mae_np(y_test_true, y_test_pred)

print(f"TEST RMSE = {test_rmse:.3f}")
print(f"TEST MAE  = {test_mae:.3f}")


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test_true, y_test_pred, alpha=0.4)
minv = min(y_test_true.min(), y_test_pred.min())
maxv = max(y_test_true.max(), y_test_pred.max())
plt.plot([minv, maxv], [minv, maxv])  # linia idealna
plt.xlabel("True")
plt.ylabel("Predicted")
plt.title("Predicted vs True (Test)")
plt.grid(True)
plt.show()


In [ ]:
errors = y_test_pred - y_test_true

plt.figure(figsize=(8,5))
plt.hist(errors, bins=40)
plt.xlabel("Prediction Error (pred - true)")
plt.ylabel("Count")
plt.title("Error Distribution (Test)")
plt.grid(True)
plt.show()


In [ ]:
df_test_analysis = pd.DataFrame({
    "y_true": y_test_true,
    "y_pred": y_test_pred,
    "error": y_test_pred - y_test_true,
    "abs_error": np.abs(y_test_pred - y_test_true)
})

display(df_test_analysis.sort_values("abs_error", ascending=False).head(15))
display(df_test_analysis.sort_values("abs_error", ascending=True).head(15))


In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(results_df["hidden_size"], results_df["best_valid_rmse"], alpha=0.6)
plt.xlabel("Hidden size")
plt.ylabel("Best Valid RMSE")
plt.title("Effect of Hidden Size on Valid RMSE")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(np.log10(results_df["lr"]), results_df["best_valid_rmse"], alpha=0.6)
plt.xlabel("log10(lr)")
plt.ylabel("Best Valid RMSE")
plt.title("Effect of Learning Rate on Valid RMSE")
plt.grid(True)
plt.show()


In [ ]:
results_df.to_csv("hyperparam_search_results.csv", index=False)
histories_df.to_csv("hyperparam_search_histories.csv", index=False)
final_history.to_csv("final_training_history.csv", index=False)

print("Saved:")
print("- hyperparam_search_results.csv")
print("- hyperparam_search_histories.csv")
print("- final_training_history.csv")
